In [50]:
import openvr


### Rough draft code:
Intended for use in a dynamic device assignment tool for `config.py/CONNECTED_DEVICES`

In [51]:
def get_device_serial_number(vr_system, device_index):
    return vr_system.getStringTrackedDeviceProperty(device_index, openvr.Prop_SerialNumber_String)

def identify_hmd(vr_system, device_index):
    hmd_serial_number = get_device_serial_number(vr_system, device_index)
    # logging
    print(f"Detected HMD: {vr_system.getStringTrackedDeviceProperty(device_index, openvr.Prop_ModelNumber_String)}")
    print(f"Serial Number: {hmd_serial_number}")

def identify_device(vr_system, device_index):
    device_class = vr_system.getTrackedDeviceClass(device_index)
    if device_class == openvr.TrackedDeviceClass_HMD:
        identify_hmd(vr_system, device_index)
        
    # if device_class == openvr.TrackedDeviceClass_Controller:

vr_isystem = openvr.init(openvr.VRApplication_Other)

# Loops to 64, read doc on openvr.k_unMaxTrackedDeviceCount
for device_index in range(openvr.k_unMaxTrackedDeviceCount):
    if vr_isystem.isTrackedDeviceConnected(device_index):
        identify_device(vr_isystem, device_index)
            

Detected HMD: Meta Quest 3
Serial Number: 1PASH5D1P17365


In [52]:
def get_all_devices(vr_system):
    devices = []
    for i in range(openvr.k_unMaxTrackedDeviceCount):
        if vr_system.isTrackedDeviceConnected(i):
            device_serial_number = vr_system.getStringTrackedDeviceProperty(i, openvr.Prop_SerialNumber_String)
            devices.append(device_serial_number)
    return devices
connected_devices = get_all_devices(vr_isystem)
connected_devices

['1PASH5D1P17365',
 'human://LEFT_FOOT',
 'human://LEFT_KNEE',
 'human://RIGHT_FOOT',
 'human://RIGHT_KNEE',
 'human://WAIST',
 'human://CHEST',
 'human://RIGHT_ELBOW',
 'human://LEFT_ELBOW',
 '1PASH5D1P17365_Controller_Left',
 '1PASH5D1P17365_Controller_Right']

In [53]:
poses = openvr.VRSystem().getDeviceToAbsoluteTrackingPose(
    openvr.TrackingUniverseStanding, 0, openvr.k_unMaxTrackedDeviceCount
)
print(poses)



In [54]:
chest_index = 6
pose = poses[6]
pose

In [55]:
matrix = pose.mDeviceToAbsoluteTracking
matrix

In [56]:
x = matrix[0][3]
y = matrix[1][3]
z = matrix[2][3]

print(f"Tracker ID: {chest_index}")
print(f"Position: x={x:.4f}, y={y:.4f}, z={z:.4f}")

Tracker ID: 6
Position: x=0.1637, y=1.1815, z=-0.5179


In [57]:
import numpy as np
from scipy.spatial.transform import Rotation as R

In [58]:
rot_matrix = [
    [matrix[0][0], matrix[0][1], matrix[0][2]],
    [matrix[1][0], matrix[1][1], matrix[1][2]],
    [matrix[2][0], matrix[2][1], matrix[2][2]],
]
rot_matrix

[[0.9968463182449341, 0.027629569172859192, 0.07439326494932175],
 [-0.021195482462644577, 0.9960756897926331, -0.08593239635229111],
 [-0.07647550851106644, 0.08408467471599579, 0.9935199022293091]]

In [59]:
r = R.from_matrix(rot_matrix)
r

In [60]:
qx, qy, qz, qw = r.as_quat()
print(f"Rotation (quaternion): x={qx:.4f}, y={qy:.4f}, z={qz:.4f}, w={qw:.4f}")

Rotation (quaternion): x=0.0426, y=0.0378, z=-0.0122, w=0.9983
